In [1]:
import glob
print(glob.glob("/share/*"))

import torch
import faiss
from torch_cluster import knn_graph
from timeseries.timeseries_transformer import TimeseriesDataloader
from torch_cluster import knn
print(torch.cuda.is_available())
directory_path = '/share/us-stock-dataset/Data/Stocks'


['/share/us-stock-dataset', '/share/Transformers1']
True


In [2]:

stocks_to_load = [
    "AAPL", "TSLA",
    "A", "GOOG", "AMZN", "PYPL", "NVDA", "AMD",
    "NFLX", "MSFT", "INTC", "CSCO", "ADBE", "CRM", "QCOM", "TXN", "AVGO",
    "INTU", "ORCL", "COST", "SBUX", "AMGN", "CHTR", "GILD", "CMCSA", "BKNG",
    "MDLZ", "FISV", "BIIB", "MU", "MCD", "AMAT", "ADP", "ILMN", "ATVI", "ISRG",
    "ADSK", "LRCX", "BIDU", "JD", "REGN", "WBA", "VRTX", "KHC", "WMT", "ZM", "MELI",
    "TMUS", "CTSH", "XLNX", "PCAR", "ALGN", "WDAY", "SIRI", "CTXS", "ADI", "EXC", "LULU",
    "MAR", "KLAC", "PAYX", "EA", "ILMN", "ALXN", "MNST", "BMRN", "EBAY", "CTAS", "VRSK",
    "IDXX", "CDNS", "NXPI", "ASML", "INCY", "KLAC", "MCHP", "SNPS", "SWKS", "VRSN",
    "WDC", "WYNN", "XLNX", "ZBRA", "ZTS", "AEP", "AIG", "ALL", "AXP", "BA", "BAC",
    "BK", "BLK", "C", "CAT", "CL", "COF", "COP", "COST", "CSCO", "CVS", "CVX",
    "DD", "DHR", "DIS", "DOW", "DUK", "EMR", "EXC", "F", "FDX", "GD", "GE", "GILD",
    "GM", "GOOG", "GOOGL", "GS", "HD", "HON", "IBM", "INTC", "JNJ", "JPM", "KHC", "KMI",
    "KO", "LLY", "LMT", "LOW", "MA", "MCD", "MDLZ", "MDT", "MET", "MMM",
    'BLND', 'BLNG', 'BLNKW', 'BLNGU', 'BLNGW', 'BLNK', 'BLPH', 'BLRX',
    'BLTE', 'BLU', 'BLUA', 'BLUE', 'BLW', 'BLX', 'BLZE', 'BMA', 'BMAC', 'BMAQ', 'BMAQR', 'BMAQU'
]


In [3]:
dataloader = TimeseriesDataloader(directory_path, stocks_to_load, add_diff=False)

Reading and merging CSV files: ['AAPL', 'TSLA', 'A', 'GOOG', 'AMZN', 'PYPL', 'NVDA', 'AMD', 'NFLX', 'MSFT', 'INTC', 'CSCO', 'ADBE', 'CRM', 'QCOM', 'TXN', 'AVGO', 'INTU', 'ORCL', 'COST', 'SBUX', 'AMGN', 'CHTR', 'GILD', 'CMCSA', 'BKNG', 'MDLZ', 'FISV', 'BIIB', 'MU', 'MCD', 'AMAT', 'ADP', 'ILMN', 'ATVI', 'ISRG', 'ADSK', 'LRCX', 'BIDU', 'JD', 'REGN', 'WBA', 'VRTX', 'KHC', 'WMT', 'ZM', 'MELI', 'TMUS', 'CTSH', 'XLNX', 'PCAR', 'ALGN', 'WDAY', 'SIRI', 'CTXS', 'ADI', 'EXC', 'LULU', 'MAR', 'KLAC', 'PAYX', 'EA', 'ILMN', 'ALXN', 'MNST', 'BMRN', 'EBAY', 'CTAS', 'VRSK', 'IDXX', 'CDNS', 'NXPI', 'ASML', 'INCY', 'KLAC', 'MCHP', 'SNPS', 'SWKS', 'VRSN', 'WDC', 'WYNN', 'XLNX', 'ZBRA', 'ZTS', 'AEP', 'AIG', 'ALL', 'AXP', 'BA', 'BAC', 'BK', 'BLK', 'C', 'CAT', 'CL', 'COF', 'COP', 'COST', 'CSCO', 'CVS', 'CVX', 'DD', 'DHR', 'DIS', 'DOW', 'DUK', 'EMR', 'EXC', 'F', 'FDX', 'GD', 'GE', 'GILD', 'GM', 'GOOG', 'GOOGL', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'JPM', 'KHC', 'KMI', 'KO', 'LLY', 'LMT', 'LOW', 'MA', 'MCD'

In [4]:
data=dataloader.get_data().transpose(0, 1).cuda()
data.shape

torch.Size([145, 5283])

In [5]:
training_data = torch.diff(data, dim=1)


In [6]:
training_data.std(dim=1)

tensor([ 0.6423,  1.2063,  1.0221,  0.5260,  0.8853,  1.1809,  0.8456,  0.7936,
         3.9333,  1.2111,  0.6086,  0.5720,  3.0230,  1.7608,  1.0608,  1.0412,
         2.4010,  2.1629,  0.5761,  1.6778,  0.5861,  1.7014,  3.4684,  0.7478,
         0.3246, 18.9487,  0.3683,  0.7767,  4.9890,  1.0479,  1.1935,  0.7096,
         1.0936,  3.4479,  0.6357,  1.6608,  1.8272,  2.8840,  2.7063,  0.5475,
         5.7229,  0.6554,  2.5938,  0.3892,  0.8354,  3.6235,  8.5105,  0.8644,
         0.6684,  0.4234,  3.7318,  1.9303,  0.5174,  1.1847,  0.3325,  2.2037,
         1.1649,  1.6634,  0.6020,  1.2465,  3.4479,  0.2984,  1.5064,  0.4124,
         1.9888,  1.0031,  2.3050,  0.7003,  1.2684,  2.4298,  1.8306,  1.6634,
         0.5031,  1.2254,  1.4197,  2.5348,  0.9783,  2.2057,  2.2940,  0.8363,
         0.5422, 10.1997,  0.7714,  1.1157,  3.0676,  0.4603,  0.5984,  4.2522,
         3.6781,  1.2954,  0.5075,  1.2315,  0.6725,  1.6778,  0.5720,  0.6773,
         0.9821,  0.8255,  0.9766,  1.12

In [7]:
def min_max_scaling(tensor):
    return (tensor - tensor.min()) / (tensor.max() - tensor.min())


In [8]:
#training_data = min_max_scaling(training_data)
print(training_data.shape)

torch.Size([145, 5282])


In [23]:
training_data[training_data<0]

tensor([-0.0716, -0.0683, -0.0133,  ..., -0.3160, -0.0651, -0.3903],
       device='cuda:0')

In [9]:
window_size = 1

def extract_sliding_windows(tensor, window_size):        
    windows = tensor.unfold(1, window_size, 1)
    return windows

sliding_windows=extract_sliding_windows(training_data, window_size)

print(sliding_windows.shape)


torch.Size([145, 5282, 1])


In [10]:
flat_sliding_windows = sliding_windows.reshape(-1, sliding_windows.size(2))
print(flat_sliding_windows.shape)


torch.Size([765890, 1])


In [11]:
flat_sliding_windows.std(dim=0)

tensor([5.4043], device='cuda:0')

In [12]:
def kmeans_sliding_windows(flat_windows, num_clusters=1000):
  
    windows_np = flat_windows.cpu().numpy()
    
    kmeans_model = faiss.Kmeans(flat_windows.size(1), num_clusters, niter=20, verbose=True)
    
    kmeans_model.train(windows_np)
    
    error = kmeans_model.obj[-1]  # Get the last objective value    

    n = flat_windows.shape[0]  # Number of data points
    mse = error / n
    rmse = mse ** 0.5

    _, cluster_assignment = kmeans_model.index.search(windows_np, 1)
    
    return torch.from_numpy(cluster_assignment.reshape(-1)), torch.from_numpy(kmeans_model.centroids), rmse

cluster_assignment, centroids, error = kmeans_sliding_windows(flat_sliding_windows)

print(centroids.shape)

print("Clustering error:", error)


Sampling a subset of 256000 / 765890 for training
Clustering 256000 points in 1D to 1000 clusters, redo 1 times, 200 iterations
  Preprocessing in 0.02 s
  Iteration 199 (177.72 s, search 176.86 s): objective=150031 imbalance=26.473 nsplit=0        
torch.Size([1000, 1])
Clustering error: 0.4425958143173282


In [15]:
def calculate_cluster_errors(flat_windows, cluster_assignment, centroids):   
    assigned_centroids = centroids[cluster_assignment.squeeze().long()].cuda()
    squared_errors = ((flat_windows - assigned_centroids) ** 2).sum(dim=1)
    mse = squared_errors.mean()
    rmse = mse ** 0.5
    return rmse

# Using the function
errors = calculate_cluster_errors(flat_sliding_windows, cluster_assignment, centroids)
print("Average error per data point:", errors)


Average error per data point: tensor(2.4675, device='cuda:0')


In [14]:
centroids.shape

torch.Size([1000, 1])